<a href="https://colab.research.google.com/github/PatrickRuan/AI_2025/blob/main/RAG_OpenAI/05_question_answering_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!cp -r drive/MyDrive/z-發布/"70 單元_2025H1"/"AI 課程發佈"/週三/RAG_OpenAI/docs .

# Question Answering

## Overview

Recall the overall workflow for retrieval augmented generation (RAG):

We discussed `Document Loading` and `Splitting` as well as `Storage` and `Retrieval`.

Let's load our vectorDB.

In [3]:
#!pip install dotenv
!pip install -U langchain-community
!pip install chromadb
!pip install tiktoken

In [4]:
#import os
import openai
#import sys
from getpass import getpass
#sys.path.append('../..')

#from dotenv import load_dotenv, find_dotenv
#_ = load_dotenv(find_dotenv()) # read local .env file

#openai.api_key  = os.environ['OPENAI_API_KEY']

openai.api_key = getpass("請輸入你的 OpenAI API 金鑰：")

請輸入你的 OpenAI API 金鑰：··········


The code below was added to assign the openai LLM version filmed until it is deprecated, currently in Sept 2023.
LLM responses can often vary, but the responses may be significantly different when using a different model version.

In [5]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-4.1-nano"
else:
    llm_name = "gpt-4.1-nano"

print(llm_name)

gpt-4.1-nano


In [6]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'
#embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)

embedding = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=openai.api_key
)
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)


<ipython-input-6-a7b8eb79f38e>:6: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(
<ipython-input-6-a7b8eb79f38e>:10: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)


In [7]:
print(vectordb._collection.count())

208


In [8]:
# 這裡會出錯，因為 openai 用的 default embedding text 已經不一樣...

question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [9]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0, openai_api_key=openai.api_key)

<ipython-input-9-c14ad9c02e56>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name=llm_name, temperature=0, openai_api_key=openai.api_key)


### RetrievalQA chain

In [10]:
from langchain.chains import RetrievalQA

In [11]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [12]:
result = qa_chain({"query": question})

<ipython-input-12-ef85128c39b8>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


In [13]:
result["result"]

'The major topics for this class include linear regression, hypothesis representation, feature engineering, and the use of different features in machine learning models. The course also covers concepts related to training sets, input variables (features), output variables (targets), and the application of algorithms to predict relationships between features and outcomes. Additionally, the class discusses the differences between linear and logistic regression, the importance of feature selection, and the underlying mathematical foundations such as hypothesis functions, derivatives, and learning algorithms.'

### Prompt

In [14]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [15]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [16]:
question = "Is probability a class topic?"

In [17]:
result = qa_chain({"query": question})

In [18]:
result["result"]

'Yes, probability is typically a class topic, especially in courses related to machine learning, statistics, or data analysis. It provides foundational concepts for understanding uncertainty, making predictions, and modeling data. Thanks for asking!'

In [19]:
result["source_documents"][0]

Document(metadata={'creationdate': '2008-07-11T11:25:05-07:00', 'page_label': '4', 'source': './MachineLearning-Lecture02.pdf', 'title': '', 'page': 3, 'creator': 'PScript5.dll Version 5.2.2', 'moddate': '2008-07-11T11:25:05-07:00', 'total_pages': 18, 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'author': ''}, page_content="may have more than one input feature. So for example, if instead of just knowing the size \nof the houses, if we know also the number of bedrooms in these houses, let's say, then, so \nif our training set also has a second feature, the number of bedrooms in the house, then \nyou may, let's say X1 denote the size and square feet. Let X have script two denote the \nnumber of bedrooms, and then I would write the hypothesis, H of X, as theta rho plus \ntheta 1X1 plus theta 2X2.  \nOkay, and sometimes when I went to take the hypothesis H, and when I went to make \nthis dependent on the theta is explicit, I'll sometimes write this as H subscript theta of X. \nAnd so t

### RetrievalQA chain types

In [20]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [21]:
result = qa_chain_mr({"query": question})

In [22]:
result["result"]

'The provided text does not mention whether probability is a class topic.'

If you wish to experiment on the `LangSmith platform` (previously known as LangChain Plus):

 * Go to [LangSmith](https://www.langchain.com/langsmith) and sign up
 * Create an API key from your account's settings
 * Use this API key in the code below   
 * uncomment the code  
 Note, the endpoint in the video differs from the one below. Use the one below.

In [ ]:
#import os
#os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
#os.environ["LANGCHAIN_API_KEY"] = "..." # replace dots with your api key

In [23]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)
result = qa_chain_mr({"query": question})
result["result"]

'The provided text does not mention whether probability is a class topic.'

In [24]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)
result = qa_chain_mr({"query": question})
result["result"]

'Yes, probability is typically a class topic. It is often covered in courses related to statistics, data science, machine learning, or mathematics. Understanding probability provides foundational concepts for modeling uncertainty, making predictions, and analyzing data—skills that are essential in many fields.\n\nIn the context of machine learning algorithms like locally weighted regression (also known as Loess), probability plays an important role. These algorithms often rely on probabilistic reasoning to handle uncertainty in data and predictions. For example, when models are trained on data, they implicitly or explicitly consider the likelihood of certain outcomes, which is grounded in probability theory. \n\nSo, while the specific example you provided focuses on non-parametric regression techniques that adapt locally to data points, the underlying principles of probability are fundamental to understanding how these models work, how they generalize, and how to evaluate their perform

### RetrievalQA limitations

QA fails to preserve conversational history.

In [25]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [26]:
question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]

'Yes, probability is a class topic.'

In [27]:
question = "why are those prerequesites needed?"
result = qa_chain({"query": question})
result["result"]

'The prerequisites are needed to understand the concepts and notation used in machine learning, particularly in linear regression. They help you grasp how features are represented, how hypotheses are formulated, and how parameters (thetas) are used to make predictions. Understanding these basics allows you to follow the learning process, interpret the notation, and apply the algorithms correctly.'

Note, The LLM response varies. Some responses **do** include a reference to probability which might be gleaned from referenced documents. The point is simply that the model does not have access to past questions or answers, this will be covered in the next section.